<h1 align='center'> বাংলায় মেশিন লার্নিংঃ মাল্টিপল ফিচার লিনিয়ার রিগ্রেশন </h1>
<p align='center'> K M Shariat Ullah <br> Student, Dept. of Electrical and Electronic Engineering, Shahjalal University of Science and Technology, Sylhet <br> E-mail: cast.shariat@gamil.com </p>

আগের নোটবুকে আমরা দেখেছিলাম যদি ফিচার কেবলমাত্র একটি হয় ($x$) তবে মডেল $\hat{y} = wx + b$। কিন্তু বাস্তবে তো কেবলমাত্র একটি ফিচার থাকা দুস্কর। কোনো একটি আউটপুটকে প্রভাবিত করতে পারে এমন ফ্যাক্টরের সংখ্যা একাধিক হবে এটাই স্বাভাবিক। যেমন নিচে আমরা একটি বাড়ির সাইজ, বেডরুমের সংখ্যা, ফ্লোরের সংখ্যা, বাড়ির বয়সের উপর ভিত্তি করে বাড়ির প্রাইজ দেখতে পাচ্ছি। আমাদেরকে এখন এই ফিচারগুলোর উপর ভিত্তি করে একটি মডেল তৈরি করতে হবে যাতে আমরা যে কোনো সাইজের বা যে কোনো বয়সের বাড়ির জন্য বা অন্য কোনো ফিচারের পার্টিকুলার মানের জন্য বাড়ির দাম কত হবে তা প্রেডিক্ট করতে পারে।

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data = pd.read_csv('multiple_feature_house.csv')
data

,size,bedroom,floor,age,price
0,2104,5,1,45,460
1,1416,3,2,40,232
2,1534,3,2,30,315
3,852,2,1,36,178


এখানে আমাদের ফিচার চারটি <br>

$x_1$ : `Size of the Home` <br>
$x_2$ : `No. of Bedrooms` <br>
$x_3$ : `No. of Floors` <br>
$x_4$ : `Age of Home` <br>

আর এখানে টার্গেট একটি <br>

$y$ : `Price in USD 1000`

যদি আমরা একটি সরলরেখৈক মডেল বানাতে চাই তাহলে সকল ফিচারের পাওয়ার হতে হবে 1 অর্থাৎ লিনিয়ার রিগ্রেশন। আগেরটার সাথে এখানে পার্থক্য হচ্ছে শুধু এ জায়গায় যে সেখানে ফিচার ছিল একটা, এখানে একাধিক। তাহলে এখানে আমাদের মডেলটি হবে - 

$$f_{w_i,b}(x_i) = w_1 x_1 + w_2 x_2 + w_3 x_3 + w_4 x_4 + b \tag{1}$$

weight দিয়ে মূলত কী বুঝায় তা নিয়ে আরেকটু ইনটুইটিভ আলোচনা করা উচিত বলে মনে হচ্ছে। weight মূলত কোন ফিচারের প্রভাব কেমন তা বুঝাতে ব্যবহার করা হয়। যেমন ধরেন আপনাদের থিওরি কোর্সের weight ল্যাব কোর্সের থেকে বেশি। weight মানে এখানে গুরুত্ববহ বেশি এটা বোঝানো হচ্ছে। তাই থিওরি কোর্সের Credit 3 আর ল্যাব কোর্সের Credit 1.5। তো যেই কোর্সের weight বেশি তার প্রভাব বেশি থাকবে বা সেটি আউটপুট তৈরিতে অন্যদের থেকে বেশি গুরুত্বপূর্ণ। যেমন উপরের চার ফিচারের জন্য আমরা পার্টিকুলারলি একটা মডেল কল্পনা করতে পারি - 

$$ f_{w_i,b}(x_i) = 0.1 x_1 + 4x_2 + 10 x_3 - 2x_4 + 80 $$

এই মডেলটি থেকে কিছু বিষয় উদ্ধার করে নেই আগে - 

<ul>
    <li>যদি কোনো বেডরুম না থাকে, কোনো ফ্লোর না থাকে, বাড়ির কোনো বয়স নেই, বাড়ির কোনো সাইজও নেই অর্থাৎ $x_1 = x_2 = x_3 = x_4 = 0$ তাহলে ঐ বাড়িটির দাম হবে ৮০ হাজার ডলার। এটাই b এর মান, তুলনা করে দেখেন</li>
    <li> বেডরুমের সংখ্যার সাথে 10 আছে ওয়েট হিসেবে। অন্যদিকে ফ্লোরের সংখ্যার সাথে 4 আছে ওয়েট হিসেবে। এ থেকে একটু তুলনা করেন। অর্থাৎ ফ্লোরের সংখ্যা যদি একটা বৃদ্ধি পায় তাহলে বাড়ির প্রাইজ চারগুণ হবে। আর যদি একটা বেডরুমের সংখ্যা বৃদ্ধি পায় তাহলে বাড়ির প্রাইজ দশগুণ বৃদ্ধি পাবে। এখানে তাহলে বেডরুমের সংখ্যাটা কততলা বাড়ি তার থেকেও বেশি গুরুত্বপূর্ণ। এজন্যই বেডরুমের সংখ্যার জন্য বড় ওয়েট নেওয়া হয়েছে। 
</ul>

#### Vectorization

যখন এধরণের একাধিক ফিচার নিয়ে মডেল তৈরি করা হবে তখন আমরা ভেক্টরাইজেশন নামে একটি পদ্ধতি অ্যাপ্লাই করব। এর জন্য আমরা দুইটা ভেক্টর $\vec{x}$ ও $\vec{w}$ সঙ্গায়িত করব। উপরের মডেলটির জন্য এ দুটি ভেক্টরকে এভাবে লেখা হবে - 

$$\vec{x} =  \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix} $$
$$\vec{w} =  \begin{bmatrix} 0.1 & 4 & 10 & -2 \end{bmatrix} $$

এমন ক্ষেত্রে আমাদের মডেলটিকে লেখা হবে এভাবে - 

$$f(\vec{w},\vec{x})=\vec{w} \cdot \vec{x} + b \tag{2}$$

ভেক্টরাইজেশন করার পর আগের পদ্ধতিতেই আমরা গ্র্যাডিয়েন্ট ডিসেন্ট করে কস্ট ফাংশনকে মিনিমাম করে এমন কিছু ওয়েট ও বায়াসের মান বের করব। 

In [4]:
x_train = np.array([
    [2104, 1416, 1534, 852],
    [5, 3, 3, 2],
    [1, 2, 2, 1],
    [45, 40, 30, 36]
])

y_train = np.array([460, 232, 315, 178])

In [10]:
b_init = 80.
w_init = np.array([0.1, 4, 10, -2])

y = np.dot(x_train, w_init) + b_init
y

array([19590.4,   118.5,   106.1,   472.5])

#### Cost Function

In [12]:
m,n = np.shape(x_train)
cost = 0
for i in range(m):
    y_hat = np.dot(x_train[i],w_init) + b_init 
    cost = cost + (y_hat - y_train[i])**2

cost = cost/(2*m)
cost 

45764431.98375

#### Gradient Descent

In [18]:
m,n = np.shape(x_train)        
dj_dw = np.zeros((n,))
dj_db = 0.
a = 0.01

for i in range(m):
    error = np.dot(x_train[i],w_init) + b_init - y_train[i]
    for j in range(n):
        dj_dw[j] = dj_dw[j] + error * x_train[i, j]
    dj_db = dj_db + error
dj_dw = dj_dw/m
dj_db = dj_db/m
w_init = w_init - a * dj_dw
b_init = b_init - a * dj_db

w_init, b_init

(array([-5.69698474e+19, -3.83472387e+19, -4.15341717e+19, -2.30810870e+19]),
 -2.779688721683911e+16)